 **変数の意味**

- state : あがりに必要な枚数の手牌の状態

- hand : stateから一枚切った状態

- n : 用いる牌の種類数

- m : 手牌の枚数

- l : 同一牌の枚数(基本的に4で固定)


In [1]:
from collections import defaultdict
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import time

random_seed = 34
np.random.seed(random_seed)

### 必要な関数の再定義

In [2]:
def is_valid(seq, l=4): # 生成された組み合わせが手牌として妥当かどうかを判断する関数　tuple(seq)の一つ一つが一つの状態(手牌)に対応している
    counts = defaultdict(lambda: 0)
    for i in range(0, len(seq)):
        if i + 1 < len(seq) and seq[i] > seq[i + 1]: # 前半の条件はiが一番最後以外は常に成立、後半の条件は昇順に整列するための条件
            return False
        counts[seq[i]] += 1
        if (counts[seq[i]] > l): return False # 牌の上限枚数を超えたらFalse
    return True

def number_state_slow(n,m,l): # 全ての手牌の組み合わせの数を出力する関数
    count = 0
    for seq in itertools.product(range(n), repeat = m): # 直積を作る関数, n=9 m=5 なら 9 ** 5 回繰り返す　
        if is_valid(seq,l):
            count += 1
            #print(list(seq))
    return count
    
def generate_all_l(n, m, l=4): # 全ての手牌の組み合わせをタプルで出力する関数
    gen_list = []
    for seq in itertools.product(range(n), repeat = m):
        if is_valid(seq, l):
            gen_list.append(seq)
    return gen_list

# 手牌(state)を、牌種ごとの枚数のリスト(長さn)に変換する関数
def state_to_hist(state, n):
    hist = [0] * n # hist = [0,0,...,0]
    for c in state:
        hist[c] += 1
    return hist

def states_to_hist(state_list, n): # 手牌(state)を、牌種ごとの枚数のリスト(長さn)に変換する関数
    hist_list = []
    for state in state_list:
        #print(state)
        ret = [0] * n # ret = [0,0,...,0]
        for c in state:
            ret[c] += 1
        hist_list.append(ret)
    return hist_list

def hand_to_prob_and_state(hand, state_nml, n, m, l=4): # ある手牌(hand)における、1枚ツモる時の遷移確率(prob)と手牌(state)のindexのタプルを出す関数
    ret = [l] * n  #  残り枚数を表すリスト
    for h in hand:
        ret[h] -= 1
    yama_sum = n * l - (m - 1)
    state_list = []
    for i in range(n):
        if ret[i] == 0: 
            continue
        prob = ret[i] / yama_sum # 遷移確率
        state = tuple(sorted(list(hand) + [i])) # 遷移後の手牌
        #print(state)
        state_index = state_nml.index(state) # 遷移後の手牌のindex
        #print(state_index)
        state_list.append((prob, state_index))
    return state_list

def state_to_hand(state): # ある手牌stateに遷移できるhandを出力する関数
    return list(set(tuple(state[:i] + state[i+1:]) for i in range(len(state)))) # i番目の要素を取り除く

def win_split_sub(hist, two, three, split_state, agari_list):
    if any(x < 0 for x in hist):
        return
    if two == 0 and three == 0:
        agari_list.append(tuple(split_state))
        return
    i = next(i for i, x in enumerate(hist) if x > 0) # histの中でx>０を満たす最小のindexを持ってくる
    next_hist = [x - 2 if i == j else x for j, x in enumerate(hist)]
    if two > 0 and hist[i] == 2: # 雀頭
        win_split_sub(next_hist, two - 1, three, split_state + [(i, i)], agari_list)
    next_hist = [x - 3 if i == j else x for j, x in enumerate(hist)]
    if three > 0 and hist[i] == 3: # 刻子
        win_split_sub(next_hist, two, three - 1, split_state + [(i, i, i)], agari_list)
    next_hist = [x -1 if i <= j <= i + 2 else x for j, x in enumerate(hist)]
    if three > 0 and i + 2 < len(hist): # 順子
        win_split_sub(next_hist, two, three - 1, split_state + [(i, i+1, i+2)], agari_list)
    return 
    
def win_split_main(hist):
    n_two = 1 if sum(hist) % 3 == 2 else 0
    n_three = sum(hist) // 3
    agari_list = []
    win_split_sub(hist, n_two, n_three, [], agari_list)
    if len(agari_list) == 0:
        return (False, set())
    else:
        return (True, agari_list)

def is_tanyao(state):
    for hai in state:
        if hai == 0 or hai == 8:
            return False
    return True

def is_chanta(split_state):
    state_value = True
    for block in split_state:
        if 0 in block or 8 in block:
            continue
        else:
            state_value = False
            break
    return state_value

def is_toitoi(split_state):
    state_value = True
    for block in split_state:
        if len(block) == 2: # 雀頭
            continue
        else:  # 面子
            if block[0] != block[1]:
                state_value = False
                break
    return state_value
    
def is_ipeko(split_state):
    for block in split_state:
        if len(block) == 2:
            continue
        if block[0] != block[1]:
            temp = list(split_state)
            temp.remove(block)
            if block in temp:
                return True
    return False

# state_nml, hand_nml, hist_nmlが必要なように書き換え
def value_iteration(state_nml, hand_nml, hist_nml, n, m, l, gamma):
    #state_nml = generate_all_l(n, m, l)
    #hand_nml = generate_all_l(n, m-1, l)
    #hist_nml = states_to_hist(state_nml, n)
    is_win_nml, split_state_list_nml = [], []
    for i, hist in enumerate(hist_nml):
        val, split = win_split_main(hist)
        is_win_nml.append(val)
        split_state_list_nml.append(split)
    #print(is_win_nml)
    #print(split_state_set_nml)
    #is_win_nml = [is_win_main(hist) for hist in hist_nml]
    h2ps_nml = [hand_to_prob_and_state(hand, state_nml, n, m, l) for hand in hand_nml]
    s2h_nml = [[hand_nml.index(hand) for hand in state_to_hand(state)] for state in state_nml]
    value_hand = [0] * len(hand_nml)
    n_hand = len(hand_nml)
    value_state = [1 if is_win_nml[i] else 0 for i in range(len(state_nml))] # あがっていればvalueは1、いなければ0
    value_state = [2 * value_state[i] if is_tanyao(state) else value_state[i] for i, state in enumerate(state_nml)] # tannyao
    # 役判定(断么九以外)
    for i, split_state_list in enumerate(split_state_list_nml):
        if len(split_state_list) == 0:
            continue
        elif len(split_state_list) == 1:
            if is_chanta(split_state_list[0]):
                value_state[i] *= 2
            if is_toitoi(split_state_list[0]):
                value_state[i] *= 2
            if is_ipeko(split_state_list[0]):
                value_state[i] *= 2
        else:
            max_state_value = value_state[i]
            for split_state in split_state_list:
                temp_state_value = value_state[i]
                if is_chanta(split_state):
                    temp_state_value *= 2
                if is_toitoi(split_state):
                    temp_state_value *= 2
                if is_ipeko(split_state):
                    temp_state_value *= 2                 

                if temp_state_value > max_state_value:
                    max_state_value = temp_state_value
            value_state[i] = max_state_value
    n_state = len(state_nml)
    theta = 1e-6
    while True:
        #print('iteration')
        delta = 0
        for i in range(n_hand):
            old_v = value_hand[i]
            value_hand[i] = sum(p * value_state[n] for (p, n) in h2ps_nml[i])
            delta = max(delta, abs(old_v - value_hand[i]))
        if delta < theta: break
        for i in range(n_state):
            if is_win_nml[i]: continue
            value_state[i] = max(gamma * value_hand[n] for n in s2h_nml[i])
    print('finish iterations')
    return value_hand # 各handのvalueをリストにして返す

def normal_value_iteration(state_nml, hand_nml, hist_nml, n, m, l, gamma):
    #state_nml = generate_all_l(n, m, l)
    #hand_nml = generate_all_l(n, m-1, l)
    #hist_nml = states_to_hist(state_nml, n)
    is_win_nml, split_state_list_nml = [], []
    for i, hist in enumerate(hist_nml):
        val, split = win_split_main(hist)
        is_win_nml.append(val)
        split_state_list_nml.append(split)
    #print(is_win_nml)
    #print(split_state_set_nml)
    #is_win_nml = [is_win_main(hist) for hist in hist_nml]
    h2ps_nml = [hand_to_prob_and_state(hand, state_nml, n, m, l) for hand in hand_nml]
    s2h_nml = [[hand_nml.index(hand) for hand in state_to_hand(state)] for state in state_nml]
    value_hand = [0] * len(hand_nml)
    n_hand = len(hand_nml)
    value_state = [1 if is_win_nml[i] else 0 for i in range(len(state_nml))] # あがっていればvalueは1、いなければ0
    n_state = len(state_nml)
    theta = 1e-6
    while True:
        #print('iteration')
        delta = 0
        for i in range(n_hand):
            old_v = value_hand[i]
            value_hand[i] = sum(p * value_state[n] for (p, n) in h2ps_nml[i])
            delta = max(delta, abs(old_v - value_hand[i]))
        if delta < theta: break
        for i in range(n_state):
            if is_win_nml[i]: continue
            value_state[i] = max(gamma * value_hand[n] for n in s2h_nml[i])
    print('finish iterations')
    return value_hand # 各valueのhandをリストにして返す

# stateとその時にvalueが最大となる捨て牌のタプルを入れたリスト max_value_discard_list = [((0, 0, 0, 0, 1), {0}), ((0, 0, 0, 0, 2), {0}), ... ,((7, 8, 8, 8, 8), {8})]
# state_nmlのうち、あがり形を抜いたもの discard_state_nml = [(0, 0, 0, 0, 1), (0, 0, 0, 0, 2), ..., (7, 8, 8, 8, 8)]
def states_to_max_value_list(state_nml, hand_nml, hist_nml, value_hand_nml, n, m, l=4):
    max_value_list = []
    discard_state_nml = []
    #hist_nml = states_to_hist(state_nml, n)
    for i, hist in enumerate(hist_nml):        
        if win_split_main(hist)[0]:
            continue # あがっているstateの時は何も入れない
        else:
            max_value = 0
            max_p = []
            for j in range(m):
                state = state_nml[i]
                hand = state[:j] + state[j+1:]
                ind = hand_nml.index(tuple(hand))
                hand_val = value_hand_nml[ind]
                if max_value < hand_val:
                    max_p = {state[j]}
                    max_value = hand_val
                elif round(max_value, 5) == round(hand_val, 5): # 小数点以下5桁まで同じなら同じとみなす
                    max_p.add(state[j])
            discard_state_nml.append(state_nml[i])
            max_value_list.append(tuple((state_nml[i], max_p)))
    return max_value_list, discard_state_nml # 正直discard_hist_nmlを出す方が早い

# 各stateにおいて、出力してほしい捨て牌の確率分布を出力する
def discard_ans_prob_vector(max_value_discard_list, n, m, l):
    discard_vector = []
    for i, discard in max_value_discard_list:
        v = [0] * n
        num = len(discard)
        for p in discard:
            v[p] = 1 / num # 答えの数で割った値を教師とする  こうしないと学習がうまくいかない?
            #v[p] = 1
        discard_vector.append(v)
    return discard_vector

### 捨て牌ベクトルの作成

In [3]:
n = 9
m = 8
l = 4

state_nml = generate_all_l(n, m, l)
hand_nml = generate_all_l(n, m - 1, l)
hist_nml = states_to_hist(state_nml, n)

#value_hand_nml06 = value_iteration(state_nml, hand_nml, hist_nml, n, m, l, 0.6)
value_hand_nml09 = value_iteration(state_nml, hand_nml, hist_nml, n, m, l, 0.9)
#value_hand_nml099 = value_iteration(state_nml, hand_nml, hist_nml, n, m, l, 0.99)

#normal_value_hand_nml06 = normal_value_iteration(state_nml, hand_nml, hist_nml, n, m, l, 0.6)
normal_value_hand_nml09 = normal_value_iteration(state_nml, hand_nml, hist_nml, n, m, l, 0.9)
#normal_value_hand_nml099 = normal_value_iteration(state_nml, hand_nml, hist_nml, n, m, l, 0.99)

finish iterations
finish iterations


In [4]:
#max_value_discard_list06, discard_state_nml = states_to_max_value_list(state_nml, hand_nml, hist_nml, value_hand_nml06, n, m, l)
max_value_discard_list09, discard_state_nml = states_to_max_value_list(state_nml, hand_nml, hist_nml, value_hand_nml09, n, m, l)
#max_value_discard_list099, discard_state_nml = states_to_max_value_list(state_nml, hand_nml, hist_nml, value_hand_nml099, n, m, l)

#normal_max_value_discard_list06, discard_state_nml =  states_to_max_value_list(state_nml, hand_nml, hist_nml, normal_value_hand_nml06, n, m, l)
normal_max_value_discard_list09, discard_state_nml =  states_to_max_value_list(state_nml, hand_nml, hist_nml, normal_value_hand_nml09, n, m, l)
#normal_max_value_discard_list099, discard_state_nml =  states_to_max_value_list(state_nml, hand_nml, hist_nml, normal_value_hand_nml099, n, m, l)

In [5]:
#np.save('max_value_discard_list06', np.array(max_value_discard_list06))
np.save('max_value_discard_list09', np.array(max_value_discard_list09))
#np.save('max_value_discard_list099', np.array(max_value_discard_list099))

#np.save('normal_max_value_discard_list06', np.array(normal_max_value_discard_list06))
np.save('normal_max_value_discard_list09', np.array(normal_max_value_discard_list09))
#np.save('normal_max_value_discard_list099', np.array(normal_max_value_discard_list099))

### 結果を見る

In [6]:
for v in sorted((hand, value_hand_nml09[i], normal_value_hand_nml09[i]) for i, hand in enumerate(hand_nml)):
    print (v)

((0, 0, 0, 0, 1, 1, 1), 1.0849632785146, 0.6973367750723577)
((0, 0, 0, 0, 1, 1, 2), 1.8918918796326243, 0.7246309856034419)
((0, 0, 0, 0, 1, 1, 3), 1.1723553832002203, 0.6545183580235789)
((0, 0, 0, 0, 1, 1, 4), 1.1663053926654776, 0.6519950092562166)
((0, 0, 0, 0, 1, 1, 5), 1.169458742303457, 0.6568514705488455)
((0, 0, 0, 0, 1, 1, 6), 1.166840864490913, 0.6598905964169361)
((0, 0, 0, 0, 1, 1, 7), 1.1653818893978962, 0.6346042060867648)
((0, 0, 0, 0, 1, 1, 8), 1.2025159909404564, 0.6120785458098912)
((0, 0, 0, 0, 1, 2, 2), 1.8918918796326243, 0.7246309856034419)
((0, 0, 0, 0, 1, 2, 3), 1.2687220037578615, 0.741486409731981)
((0, 0, 0, 0, 1, 2, 4), 1.2687220037578615, 0.719707089205553)
((0, 0, 0, 0, 1, 2, 5), 1.2687220037578615, 0.7105429943599149)
((0, 0, 0, 0, 1, 2, 6), 1.2687220037578615, 0.7100152909639701)
((0, 0, 0, 0, 1, 2, 7), 1.2687220037578615, 0.6961360587490051)
((0, 0, 0, 0, 1, 2, 8), 1.474201455812656, 0.6848277990487752)
((0, 0, 0, 0, 1, 3, 3), 1.0524813906953916, 0.67

((1, 1, 1, 1, 2, 2, 5), 1.7327553631870065, 0.7004097845492616)
((1, 1, 1, 1, 2, 2, 6), 1.6912315915815634, 0.7056140363432298)
((1, 1, 1, 1, 2, 2, 7), 1.6708407689749132, 0.6874550834613222)
((1, 1, 1, 1, 2, 2, 8), 1.5310209612449714, 0.6762168593415304)
((1, 1, 1, 1, 2, 3, 3), 1.8308264358713888, 0.8195287791069303)
((1, 1, 1, 1, 2, 3, 4), 1.5331671170593582, 0.7758818519706564)
((1, 1, 1, 1, 2, 3, 5), 1.5712328202151777, 0.7637499006894689)
((1, 1, 1, 1, 2, 3, 6), 1.5458917893721071, 0.7499747224730857)
((1, 1, 1, 1, 2, 3, 7), 1.5099837817729203, 0.7417921298842457)
((1, 1, 1, 1, 2, 3, 8), 1.3804660572288554, 0.736140016961532)
((1, 1, 1, 1, 2, 4, 4), 1.6221113355896724, 0.8022583908118115)
((1, 1, 1, 1, 2, 4, 5), 1.4744498874826912, 0.7237926474736173)
((1, 1, 1, 1, 2, 4, 6), 1.455930233085878, 0.6924201754980588)
((1, 1, 1, 1, 2, 4, 7), 1.4242620902395515, 0.6646338028180994)
((1, 1, 1, 1, 2, 4, 8), 1.3297736977725536, 0.6609292033417171)
((1, 1, 1, 1, 2, 5, 5), 1.642747583981756,

((2, 2, 3, 4, 4, 5, 5), 2.280294919927594, 0.7934202116055853)
((2, 2, 3, 4, 4, 5, 6), 1.7782006261734902, 0.7538802184125039)
((2, 2, 3, 4, 4, 5, 7), 1.7464270562958026, 0.6800906488356631)
((2, 2, 3, 4, 4, 5, 8), 1.6658309147854524, 0.6653859902974719)
((2, 2, 3, 4, 4, 6, 6), 2.2993004423149888, 0.7075431150868804)
((2, 2, 3, 4, 4, 6, 7), 1.7479633908600125, 0.6748791047886644)
((2, 2, 3, 4, 4, 6, 8), 1.6766312094227946, 0.644918018424627)
((2, 2, 3, 4, 4, 7, 7), 2.2993004383267865, 0.7126759400508897)
((2, 2, 3, 4, 4, 7, 8), 1.6633492870245274, 0.6309549402691333)
((2, 2, 3, 4, 4, 8, 8), 1.5223600709689349, 0.7049769983691222)
((2, 2, 3, 4, 5, 5, 5), 1.7675753361777542, 0.7957732487935572)
((2, 2, 3, 4, 5, 5, 6), 1.8095261872419979, 0.8052441496575518)
((2, 2, 3, 4, 5, 5, 7), 1.739192471769981, 0.7417503560407644)
((2, 2, 3, 4, 5, 5, 8), 1.6480765006734566, 0.7010237846169154)
((2, 2, 3, 4, 5, 6, 6), 1.816554975092638, 0.7465460335639892)
((2, 2, 3, 4, 5, 6, 7), 1.543955425920817, 0

In [6]:
for v in sorted((value_hand_nml09[i], hand) for i, hand in enumerate(hand_nml)):
    print (v)

(0.9593516667067804, (0, 4, 5, 8, 8, 8, 8))
(0.9593516667067805, (0, 0, 0, 0, 3, 4, 8))
(0.9596956151756563, (0, 0, 0, 0, 3, 5, 8))
(0.9596956151756563, (0, 3, 5, 8, 8, 8, 8))
(0.9600129803036843, (0, 0, 0, 0, 4, 5, 8))
(0.9600129803036843, (0, 3, 4, 8, 8, 8, 8))
(0.9804249449447682, (0, 0, 0, 0, 4, 7, 8))
(0.9804249449447682, (0, 1, 4, 8, 8, 8, 8))
(0.9814422445504125, (0, 0, 0, 0, 5, 7, 8))
(0.9814422445504125, (0, 1, 3, 8, 8, 8, 8))
(0.984135360171166, (0, 1, 5, 8, 8, 8, 8))
(0.9841353601711662, (0, 0, 0, 0, 3, 7, 8))
(0.9848537188058307, (0, 0, 0, 0, 4, 5, 6))
(0.9848537188058307, (2, 3, 4, 8, 8, 8, 8))
(0.9851463186742243, (0, 0, 0, 0, 3, 6, 8))
(0.9851463186742243, (0, 2, 5, 8, 8, 8, 8))
(0.9852802251449341, (0, 0, 0, 0, 4, 6, 8))
(0.9852802251449342, (0, 2, 4, 8, 8, 8, 8))
(0.9854869261525893, (1, 3, 4, 8, 8, 8, 8))
(0.9854869261525894, (0, 0, 0, 0, 4, 5, 7))
(0.9861437546754421, (0, 0, 0, 0, 5, 6, 8))
(0.9861437546754421, (0, 2, 3, 8, 8, 8, 8))
(0.9871888874058262, (2, 4, 5, 8,

(1.3625936932468439, (0, 3, 5, 7, 7, 7, 8))
(1.3628037465145895, (0, 1, 2, 3, 4, 5, 5))
(1.3628037465145897, (3, 3, 4, 5, 6, 7, 8))
(1.3628320483539322, (0, 1, 3, 4, 5, 5, 8))
(1.3628320483539325, (0, 3, 3, 4, 5, 7, 8))
(1.3628320483539325, (1, 3, 4, 5, 5, 8, 8))
(1.3628320483539327, (0, 0, 3, 3, 4, 5, 7))
(1.3630169877672134, (0, 0, 1, 3, 4, 4, 5))
(1.3630169877672138, (3, 4, 4, 5, 7, 8, 8))
(1.363161572223514, (0, 3, 4, 5, 5, 6, 8))
(1.3631615722235142, (0, 2, 3, 3, 4, 5, 8))
(1.3642606290503156, (0, 0, 3, 3, 6, 6, 7))
(1.3642606290503159, (1, 2, 2, 5, 5, 8, 8))
(1.3642858619550118, (0, 2, 3, 4, 4, 6, 8))
(1.364285861955012, (0, 2, 4, 4, 5, 6, 8))
(1.365156428569846, (0, 0, 5, 6, 7, 8, 8))
(1.3651564285698463, (0, 0, 1, 2, 3, 8, 8))
(1.3651825841241085, (0, 2, 2, 6, 6, 7, 8))
(1.3651825841241088, (0, 1, 2, 2, 6, 6, 8))
(1.365832882558487, (0, 0, 3, 5, 5, 5, 6))
(1.365832882558487, (0, 2, 3, 3, 3, 5, 8))
(1.365832882558487, (0, 3, 5, 5, 5, 6, 8))
(1.3658328825584871, (2, 3, 3, 3, 5, 8

(1.491032952293812, (0, 2, 2, 4, 6, 7, 7))
(1.491032952293812, (2, 2, 4, 6, 7, 7, 8))
(1.4910329522938122, (0, 1, 1, 2, 4, 6, 6))
(1.4910329522938122, (1, 1, 2, 4, 6, 6, 8))
(1.491928788551784, (2, 3, 3, 4, 5, 6, 8))
(1.4919287885517842, (0, 2, 3, 4, 5, 5, 6))
(1.4923728441129016, (0, 2, 2, 7, 7, 7, 7))
(1.4923728441129018, (1, 1, 1, 1, 6, 6, 8))
(1.4926899188289042, (1, 1, 1, 1, 4, 5, 6))
(1.4926899188289042, (2, 3, 4, 7, 7, 7, 7))
(1.4929685207006034, (3, 4, 6, 6, 7, 7, 8))
(1.4929685207006036, (0, 1, 1, 2, 2, 4, 5))
(1.4933319121568431, (0, 2, 3, 3, 3, 3, 5))
(1.4933319121568431, (0, 3, 5, 5, 5, 5, 6))
(1.4933319121568431, (3, 5, 5, 5, 5, 6, 8))
(1.4933319121568434, (2, 3, 3, 3, 3, 5, 8))
(1.4937948386631466, (0, 1, 1, 1, 2, 3, 6))
(1.4937948386631468, (2, 5, 6, 7, 7, 7, 8))
(1.4951996806667733, (0, 1, 2, 2, 5, 6, 6))
(1.4951996806667733, (0, 2, 2, 3, 6, 6, 7))
(1.4951996806667733, (1, 2, 2, 5, 6, 6, 8))
(1.4951996806667736, (2, 2, 3, 6, 6, 7, 8))
(1.4952001703205944, (1, 2, 3, 5, 5

(2.3342593436827577, (2, 2, 3, 3, 4, 4, 5))
(2.3342593436827577, (2, 3, 3, 4, 4, 5, 5))
(2.3342593436827577, (3, 4, 4, 5, 5, 6, 6))
(2.3342593436827577, (4, 4, 5, 5, 6, 6, 7))
(2.334259343682758, (3, 3, 4, 4, 5, 5, 6))
(2.3662273756507544, (3, 3, 4, 4, 5, 5, 7))
(2.366227375650755, (1, 3, 3, 4, 4, 5, 5))
(2.366227375650755, (1, 4, 4, 5, 5, 6, 6))
(2.366227375650755, (2, 2, 3, 3, 4, 4, 6))
(2.366227375650755, (2, 4, 4, 5, 5, 6, 6))
(2.3662273756507552, (2, 2, 3, 3, 4, 4, 7))
(2.4615382440601663, (1, 1, 1, 2, 2, 5, 5))
(2.4615382440601663, (1, 1, 1, 2, 2, 6, 6))
(2.4615382440601663, (1, 1, 1, 2, 2, 7, 7))
(2.4615382440601663, (1, 1, 1, 3, 3, 6, 6))
(2.4615382440601663, (1, 1, 1, 3, 3, 7, 7))
(2.4615382440601663, (1, 1, 1, 4, 4, 6, 6))
(2.4615382440601663, (1, 1, 1, 4, 4, 7, 7))
(2.4615382440601663, (1, 1, 1, 5, 5, 7, 7))
(2.4615382440601663, (1, 1, 2, 2, 2, 4, 4))
(2.4615382440601663, (1, 1, 2, 2, 2, 5, 5))
(2.4615382440601663, (1, 1, 2, 2, 2, 6, 6))
(2.4615382440601663, (1, 1, 2, 2, 2, 

In [7]:
for v in sorted((normal_value_hand_nml09[i], hand) for i, hand in enumerate(hand_nml)):
    print (v)

(0.5470624372388464, (0, 0, 1, 1, 4, 7, 8))
(0.5470624372388465, (0, 1, 4, 7, 7, 8, 8))
(0.5510600299348568, (0, 0, 1, 1, 7, 8, 8))
(0.5510600299348569, (0, 0, 1, 7, 7, 8, 8))
(0.5510842578343551, (0, 0, 1, 4, 7, 8, 8))
(0.5534519028643601, (0, 1, 3, 7, 7, 8, 8))
(0.5534519028643602, (0, 0, 1, 1, 5, 7, 8))
(0.5539913367018373, (0, 0, 1, 1, 4, 8, 8))
(0.5539913367018373, (0, 0, 4, 7, 7, 8, 8))
(0.5556748468010083, (0, 0, 5, 7, 7, 8, 8))
(0.5556748468010084, (0, 0, 1, 1, 3, 8, 8))
(0.5565655426011888, (0, 0, 1, 3, 3, 8, 8))
(0.5565655426011888, (0, 0, 5, 5, 7, 8, 8))
(0.5573098450522139, (0, 0, 1, 1, 7, 7, 8))
(0.5573098450522139, (0, 1, 1, 7, 7, 8, 8))
(0.5576078950173683, (0, 0, 1, 5, 7, 7, 8))
(0.5576078950173683, (0, 1, 1, 3, 7, 8, 8))
(0.5578683418700339, (0, 0, 1, 1, 5, 8, 8))
(0.557868341870034, (0, 0, 3, 7, 7, 8, 8))
(0.5585601658697771, (0, 1, 1, 4, 7, 8, 8))
(0.5585601658697772, (0, 0, 1, 4, 7, 7, 8))
(0.559608130396204, (0, 0, 1, 1, 3, 3, 8))
(0.559608130396204, (0, 5, 5, 7, 7

(0.6666419703034561, (1, 2, 3, 3, 5, 5, 7))
(0.6666419703034561, (1, 3, 3, 5, 5, 6, 7))
(0.6666836897922506, (0, 0, 1, 4, 5, 6, 6))
(0.6666836897922507, (2, 2, 3, 4, 7, 8, 8))
(0.666761206910383, (0, 2, 2, 5, 8, 8, 8))
(0.6667612069103831, (0, 0, 0, 3, 6, 6, 8))
(0.6667835472988148, (3, 4, 4, 7, 7, 7, 7))
(0.666783547298815, (1, 1, 1, 1, 4, 4, 5))
(0.6668051581542768, (1, 4, 7, 7, 8, 8, 8))
(0.666805158154277, (0, 0, 0, 1, 1, 4, 7))
(0.6669026988374391, (0, 0, 0, 2, 3, 7, 8))
(0.6669026988374394, (0, 1, 5, 6, 8, 8, 8))
(0.6669217810736335, (1, 1, 4, 4, 4, 4, 7))
(0.6669217810736336, (1, 4, 4, 4, 4, 7, 7))
(0.6669427527195972, (2, 3, 3, 7, 7, 7, 7))
(0.6669427527195975, (1, 1, 1, 1, 5, 5, 6))
(0.6669567011441753, (1, 1, 1, 1, 4, 7, 7))
(0.6669567011441753, (1, 1, 4, 7, 7, 7, 7))
(0.667001669419592, (1, 3, 3, 5, 7, 7, 7))
(0.6670016694195922, (1, 1, 1, 3, 5, 5, 7))
(0.6670242748735924, (0, 2, 4, 6, 6, 6, 6))
(0.6670242748735924, (2, 2, 2, 2, 4, 6, 8))
(0.6670662856973779, (0, 1, 3, 4, 4,

(0.7044218272558822, (3, 5, 5, 5, 6, 7, 7))
(0.7045110297234679, (0, 0, 3, 3, 3, 5, 5))
(0.7045110297234681, (3, 3, 5, 5, 5, 8, 8))
(0.7045154115228509, (0, 1, 3, 3, 4, 5, 6))
(0.7045154115228511, (2, 3, 4, 5, 5, 7, 8))
(0.704527978513901, (1, 5, 5, 6, 6, 6, 6))
(0.704527978513901, (2, 2, 2, 2, 3, 3, 7))
(0.7048438150494137, (1, 2, 2, 3, 3, 3, 5))
(0.7048438150494137, (3, 5, 5, 5, 6, 6, 7))
(0.7048663268473403, (0, 0, 0, 3, 3, 6, 8))
(0.7048663268473403, (0, 2, 5, 5, 8, 8, 8))
(0.7049769983691221, (0, 0, 4, 4, 5, 6, 6))
(0.7049769983691222, (2, 2, 3, 4, 4, 8, 8))
(0.7050032620344613, (1, 2, 4, 6, 6, 6, 8))
(0.7050032620344615, (0, 2, 2, 2, 4, 6, 7))
(0.70505448443655, (1, 1, 3, 3, 3, 5, 7))
(0.7050544844365502, (1, 3, 5, 5, 5, 7, 7))
(0.7051800027077751, (0, 0, 4, 4, 4, 6, 6))
(0.7051800027077753, (2, 2, 4, 4, 4, 8, 8))
(0.7051939169494307, (0, 0, 0, 4, 6, 8, 8))
(0.7051939169494307, (0, 0, 2, 4, 8, 8, 8))
(0.7052309510937717, (3, 3, 4, 6, 6, 7, 8))
(0.7052309510937718, (0, 1, 2, 2, 4,

(0.7885742127154627, (2, 3, 4, 6, 8, 8, 8))
(0.7885742127154628, (0, 0, 0, 1, 7, 7, 7))
(0.7885742127556418, (1, 2, 2, 3, 4, 7, 7))
(0.7885742127556419, (1, 1, 4, 5, 6, 6, 7))
(0.7885742127558182, (0, 0, 3, 4, 5, 5, 6))
(0.7885742127558184, (0, 0, 3, 4, 4, 5, 6))
(0.7885742127558184, (2, 3, 3, 4, 5, 8, 8))
(0.7885742127558184, (2, 3, 4, 4, 5, 8, 8))
(0.7887430555273397, (5, 5, 5, 7, 8, 8, 8))
(0.7887430555273398, (0, 0, 0, 1, 3, 3, 3))
(0.7888511313257727, (0, 1, 1, 1, 7, 7, 7))
(0.7888511313257727, (1, 1, 1, 7, 7, 7, 8))
(0.7890453453954548, (0, 0, 1, 2, 5, 5, 5))
(0.7890453453954549, (3, 3, 3, 6, 7, 8, 8))
(0.7893435151332503, (0, 0, 0, 3, 3, 3, 5))
(0.7893435151332504, (0, 0, 0, 2, 7, 7, 7))
(0.7893435151332504, (1, 1, 1, 3, 7, 7, 7))
(0.7893435151332504, (1, 1, 1, 5, 7, 7, 7))
(0.7893435151332504, (1, 1, 1, 6, 8, 8, 8))
(0.7893435151332504, (3, 5, 5, 5, 8, 8, 8))
(0.7896747531053525, (0, 0, 1, 2, 6, 6, 6))
(0.7896747531053525, (2, 2, 2, 6, 7, 8, 8))
(0.7904903080823813, (0, 1, 1, 1

In [8]:
for v in sorted((normal_value_hand_nml09[i]-value_hand_nml09[i], hand) for i, hand in enumerate(hand_nml)):
    print (v)

(-1.992118281094028, (5, 5, 6, 6, 6, 7, 7))
(-1.9921182810940272, (1, 1, 2, 2, 2, 3, 3))
(-1.9756529723644218, (3, 3, 4, 4, 4, 5, 5))
(-1.97441332603076, (4, 4, 5, 5, 5, 6, 6))
(-1.9744133260307595, (2, 2, 3, 3, 3, 4, 4))
(-1.758328614334722, (1, 1, 5, 5, 5, 7, 7))
(-1.7583286143347212, (1, 1, 3, 3, 3, 7, 7))
(-1.7574489388439085, (1, 1, 1, 6, 6, 7, 7))
(-1.757448938843908, (1, 1, 2, 2, 7, 7, 7))
(-1.7562130997476282, (1, 1, 3, 3, 7, 7, 7))
(-1.7562130997476277, (1, 1, 1, 5, 5, 7, 7))
(-1.7527841713204142, (1, 1, 1, 5, 5, 6, 6))
(-1.7527841713204118, (2, 2, 3, 3, 7, 7, 7))
(-1.7523354536147435, (1, 1, 4, 4, 4, 7, 7))
(-1.750355524169806, (2, 2, 2, 6, 6, 7, 7))
(-1.7503555241698054, (1, 1, 2, 2, 6, 6, 6))
(-1.749659926222353, (1, 1, 3, 3, 3, 4, 4))
(-1.749659926222353, (4, 4, 5, 5, 5, 7, 7))
(-1.748712157839178, (3, 3, 3, 6, 6, 7, 7))
(-1.748712157839177, (1, 1, 2, 2, 5, 5, 5))
(-1.7482717474966387, (3, 3, 4, 4, 7, 7, 7))
(-1.7482717474966374, (1, 1, 1, 4, 4, 5, 5))
(-1.7468759549957622

(-0.8069269763300331, (0, 4, 4, 4, 4, 5, 6))
(-0.8069269763300329, (2, 3, 4, 4, 4, 4, 8))
(-0.8067353516158509, (1, 1, 5, 5, 6, 8, 8))
(-0.8067353516158506, (0, 0, 2, 3, 3, 7, 7))
(-0.8066397749840543, (0, 0, 3, 3, 5, 6, 6))
(-0.8066397749840543, (2, 2, 3, 5, 5, 8, 8))
(-0.806367502324185, (0, 1, 2, 2, 2, 3, 7))
(-0.8063675023241849, (1, 5, 6, 6, 6, 7, 8))
(-0.8060834408012995, (6, 6, 7, 7, 8, 8, 8))
(-0.8060834408012991, (0, 0, 0, 1, 1, 2, 2))
(-0.8060084087682446, (1, 3, 3, 3, 5, 6, 8))
(-0.8060084087682444, (0, 2, 3, 5, 5, 5, 7))
(-0.8058280050583938, (0, 1, 1, 4, 4, 6, 8))
(-0.8058280050583937, (0, 2, 4, 4, 7, 7, 8))
(-0.8056865353993835, (0, 2, 2, 2, 2, 4, 4))
(-0.8056865353993833, (4, 4, 6, 6, 6, 6, 8))
(-0.8054697590474077, (0, 3, 3, 3, 3, 4, 5))
(-0.8054697590474074, (3, 4, 5, 5, 5, 5, 8))
(-0.8054610273998049, (1, 2, 4, 5, 5, 6, 7))
(-0.8054610273998045, (1, 2, 3, 3, 4, 6, 7))
(-0.8053398873535321, (0, 0, 1, 1, 4, 8, 8))
(-0.8053398873535317, (0, 0, 4, 7, 7, 8, 8))
(-0.8053273

(-0.5212819947929698, (4, 5, 6, 6, 6, 8, 8))
(-0.5209837318104149, (0, 0, 0, 1, 2, 5, 8))
(-0.5209837318104148, (0, 3, 6, 7, 8, 8, 8))
(-0.5208634474263225, (0, 4, 4, 4, 6, 7, 8))
(-0.5208634474263224, (0, 1, 2, 4, 4, 4, 8))
(-0.5206431438367086, (0, 0, 1, 3, 4, 6, 7))
(-0.5206431438367085, (1, 2, 4, 5, 7, 8, 8))
(-0.5202692992045529, (0, 0, 5, 5, 6, 8, 8))
(-0.5202692992045527, (0, 0, 2, 3, 3, 8, 8))
(-0.5187328536857357, (1, 2, 5, 5, 6, 7, 8))
(-0.5187328536857352, (0, 1, 2, 3, 3, 6, 7))
(-0.5179823191831606, (0, 1, 3, 4, 7, 8, 8))
(-0.5179823191831604, (0, 0, 1, 4, 5, 7, 8))
(-0.5178370251766415, (5, 7, 7, 8, 8, 8, 8))
(-0.5178370251766413, (0, 0, 0, 0, 1, 1, 3))
(-0.5167126237385511, (0, 0, 1, 1, 1, 6, 8))
(-0.5167126237385509, (0, 2, 7, 7, 7, 8, 8))
(-0.5165793250253352, (0, 3, 4, 5, 6, 7, 8))
(-0.5165793250253351, (0, 1, 2, 3, 4, 5, 8))
(-0.5162402829101743, (0, 1, 2, 4, 4, 7, 8))
(-0.5162402829101742, (0, 1, 4, 4, 6, 7, 8))
(-0.5160010542126418, (0, 0, 1, 4, 8, 8, 8))
(-0.516001

In [19]:
# ある手牌において役ありと役なしで最善手が違うのは何個あるか、その時に価値の差がもっとも大きいのはどれか
count = 0
max_diff = 0
max_state = 0
max_yaku = -1
max_simple = -1
for state in discard_state_nml:
    yaku_max = 0
    simple_max = 0
    yaku_pai = -1
    simple_pai = -1
    for i in range(len(state)):
        hand = list(state[:i]) + list(state[i+1:])
        ind = hand_nml.index(tuple(hand))
        yaku_value = value_hand_nml09[ind]
        simple_value = normal_value_hand_nml09[ind]
        if yaku_value > yaku_max:
            yaku_max = yaku_value
            yaku_pai = state[i]
        if simple_value > simple_max:
            simple_max = simple_value
            simple_pai = state[i]
    if simple_pai != yaku_pai:
        count += 1
        if yaku_value - simple_value > max_diff:
            max_diff = yaku_value - simple_value
            max_state = state
        print(state, simple_pai, yaku_pai)
print(count)
print(max_diff)
print(max_state)

(0, 0, 0, 0, 1, 1, 2, 3) 0 3
(0, 0, 0, 0, 1, 1, 2, 4) 0 4
(0, 0, 0, 0, 1, 1, 3, 4) 0 4
(0, 0, 0, 0, 1, 1, 3, 5) 0 5
(0, 0, 0, 0, 1, 1, 3, 7) 0 7
(0, 0, 0, 0, 1, 1, 3, 8) 0 3
(0, 0, 0, 0, 1, 1, 4, 5) 0 4
(0, 0, 0, 0, 1, 1, 4, 6) 0 4
(0, 0, 0, 0, 1, 1, 4, 7) 0 7
(0, 0, 0, 0, 1, 1, 4, 8) 0 4
(0, 0, 0, 0, 1, 1, 5, 6) 0 6
(0, 0, 0, 0, 1, 1, 5, 7) 0 7
(0, 0, 0, 0, 1, 1, 5, 8) 8 5
(0, 0, 0, 0, 1, 1, 6, 7) 0 7
(0, 0, 0, 0, 1, 1, 6, 8) 0 6
(0, 0, 0, 0, 1, 1, 7, 8) 0 7
(0, 0, 0, 0, 1, 2, 2, 3) 0 3
(0, 0, 0, 0, 1, 2, 2, 4) 0 4
(0, 0, 0, 0, 1, 2, 3, 4) 0 3
(0, 0, 0, 0, 1, 2, 3, 5) 0 3
(0, 0, 0, 0, 1, 2, 3, 6) 0 3
(0, 0, 0, 0, 1, 2, 3, 7) 0 3
(0, 0, 0, 0, 1, 2, 3, 8) 8 3
(0, 0, 0, 0, 1, 2, 4, 5) 0 4
(0, 0, 0, 0, 1, 2, 4, 6) 0 4
(0, 0, 0, 0, 1, 2, 4, 7) 7 4
(0, 0, 0, 0, 1, 2, 4, 8) 8 4
(0, 0, 0, 0, 1, 2, 5, 6) 0 5
(0, 0, 0, 0, 1, 2, 5, 7) 0 5
(0, 0, 0, 0, 1, 2, 5, 8) 8 5
(0, 0, 0, 0, 1, 2, 6, 7) 0 6
(0, 0, 0, 0, 1, 2, 6, 8) 0 6
(0, 0, 0, 0, 1, 2, 7, 8) 0 7
(0, 0, 0, 0, 1, 3, 3, 5) 5 0
(0, 0, 0, 0, 1

(0, 0, 0, 3, 7, 7, 7, 8) 3 8
(0, 0, 0, 4, 4, 4, 4, 5) 4 0
(0, 0, 0, 4, 4, 4, 4, 6) 4 0
(0, 0, 0, 4, 4, 4, 4, 7) 4 0
(0, 0, 0, 4, 4, 4, 5, 7) 7 0
(0, 0, 0, 4, 4, 4, 5, 8) 8 0
(0, 0, 0, 4, 4, 4, 6, 7) 7 6
(0, 0, 0, 4, 4, 4, 7, 8) 7 8
(0, 0, 0, 4, 4, 5, 5, 7) 5 0
(0, 0, 0, 4, 4, 5, 6, 6) 4 0
(0, 0, 0, 4, 4, 5, 6, 8) 8 0
(0, 0, 0, 4, 4, 5, 7, 7) 4 0
(0, 0, 0, 4, 4, 5, 7, 8) 8 0
(0, 0, 0, 4, 4, 5, 8, 8) 4 5
(0, 0, 0, 4, 4, 6, 6, 7) 6 7
(0, 0, 0, 4, 4, 6, 7, 7) 7 0
(0, 0, 0, 4, 5, 5, 5, 5) 5 0
(0, 0, 0, 4, 5, 5, 5, 7) 7 0
(0, 0, 0, 4, 5, 5, 5, 8) 8 0
(0, 0, 0, 4, 5, 5, 6, 7) 5 0
(0, 0, 0, 4, 5, 5, 6, 8) 5 0
(0, 0, 0, 4, 5, 5, 7, 7) 5 0
(0, 0, 0, 4, 5, 5, 7, 8) 4 0
(0, 0, 0, 4, 5, 5, 8, 8) 5 4
(0, 0, 0, 4, 5, 6, 6, 8) 8 0
(0, 0, 0, 4, 5, 6, 7, 8) 8 0
(0, 0, 0, 4, 5, 7, 7, 8) 8 0
(0, 0, 0, 4, 5, 7, 8, 8) 7 5
(0, 0, 0, 4, 5, 8, 8, 8) 0 4
(0, 0, 0, 4, 6, 6, 6, 7) 4 0
(0, 0, 0, 4, 6, 6, 6, 8) 8 0
(0, 0, 0, 4, 6, 6, 7, 7) 6 4
(0, 0, 0, 4, 6, 6, 7, 8) 6 0
(0, 0, 0, 4, 6, 6, 8, 8) 6 4
(0, 0, 0, 4, 6

(0, 0, 1, 2, 3, 5, 5, 7) 7 0
(0, 0, 1, 2, 3, 5, 5, 8) 8 0
(0, 0, 1, 2, 3, 5, 6, 6) 6 0
(0, 0, 1, 2, 3, 5, 6, 8) 8 0
(0, 0, 1, 2, 3, 5, 7, 7) 5 0
(0, 0, 1, 2, 3, 5, 7, 8) 8 0
(0, 0, 1, 2, 3, 5, 8, 8) 0 3
(0, 0, 1, 2, 3, 6, 6, 7) 6 0
(0, 0, 1, 2, 3, 6, 6, 8) 8 0
(0, 0, 1, 2, 3, 6, 7, 7) 7 0
(0, 0, 1, 2, 3, 6, 8, 8) 8 3
(0, 0, 1, 2, 3, 7, 7, 8) 8 0
(0, 0, 1, 2, 3, 7, 8, 8) 7 3
(0, 0, 1, 2, 4, 4, 4, 4) 4 0
(0, 0, 1, 2, 4, 4, 4, 5) 0 1
(0, 0, 1, 2, 4, 4, 4, 7) 7 0
(0, 0, 1, 2, 4, 4, 4, 8) 8 0
(0, 0, 1, 2, 4, 4, 5, 8) 8 0
(0, 0, 1, 2, 4, 4, 6, 7) 0 1
(0, 0, 1, 2, 4, 4, 7, 7) 0 1
(0, 0, 1, 2, 4, 4, 8, 8) 0 4
(0, 0, 1, 2, 4, 5, 5, 5) 0 2
(0, 0, 1, 2, 4, 5, 5, 6) 5 0
(0, 0, 1, 2, 4, 5, 5, 8) 8 0
(0, 0, 1, 2, 4, 5, 6, 6) 0 1
(0, 0, 1, 2, 4, 5, 6, 8) 8 0
(0, 0, 1, 2, 4, 5, 7, 7) 0 1
(0, 0, 1, 2, 4, 5, 7, 8) 8 0
(0, 0, 1, 2, 4, 5, 8, 8) 0 4
(0, 0, 1, 2, 4, 6, 6, 8) 8 0
(0, 0, 1, 2, 4, 6, 8, 8) 0 4
(0, 0, 1, 2, 4, 7, 7, 7) 4 0
(0, 0, 1, 2, 4, 7, 7, 8) 8 0
(0, 0, 1, 2, 4, 7, 8, 8) 7 4
(0, 0, 1, 2, 5

(0, 0, 2, 2, 4, 5, 6, 7) 2 0
(0, 0, 2, 2, 4, 5, 6, 8) 8 0
(0, 0, 2, 2, 4, 5, 7, 8) 8 0
(0, 0, 2, 2, 4, 5, 8, 8) 0 4
(0, 0, 2, 2, 4, 6, 6, 6) 2 0
(0, 0, 2, 2, 4, 6, 6, 7) 6 0
(0, 0, 2, 2, 4, 6, 6, 8) 8 0
(0, 0, 2, 2, 4, 6, 7, 7) 7 0
(0, 0, 2, 2, 4, 6, 7, 8) 2 8
(0, 0, 2, 2, 4, 6, 8, 8) 2 4
(0, 0, 2, 2, 4, 7, 7, 7) 2 0
(0, 0, 2, 2, 4, 7, 7, 8) 8 0
(0, 0, 2, 2, 4, 7, 8, 8) 7 4
(0, 0, 2, 2, 4, 8, 8, 8) 2 4
(0, 0, 2, 2, 5, 5, 5, 5) 5 0
(0, 0, 2, 2, 5, 5, 5, 6) 6 0
(0, 0, 2, 2, 5, 5, 5, 7) 7 0
(0, 0, 2, 2, 5, 5, 6, 7) 5 0
(0, 0, 2, 2, 5, 5, 6, 8) 8 0
(0, 0, 2, 2, 5, 5, 7, 8) 8 0
(0, 0, 2, 2, 5, 5, 8, 8) 0 5
(0, 0, 2, 2, 5, 6, 6, 6) 5 0
(0, 0, 2, 2, 5, 6, 6, 7) 6 0
(0, 0, 2, 2, 5, 6, 6, 8) 8 0
(0, 0, 2, 2, 5, 6, 7, 7) 7 0
(0, 0, 2, 2, 5, 6, 7, 8) 8 0
(0, 0, 2, 2, 5, 6, 8, 8) 0 5
(0, 0, 2, 2, 5, 7, 7, 7) 5 0
(0, 0, 2, 2, 5, 7, 8, 8) 0 5
(0, 0, 2, 2, 6, 6, 6, 7) 7 0
(0, 0, 2, 2, 6, 6, 7, 8) 6 0
(0, 0, 2, 2, 6, 6, 8, 8) 0 2
(0, 0, 2, 2, 6, 7, 7, 7) 6 0
(0, 0, 2, 2, 6, 7, 7, 8) 7 0
(0, 0, 2, 2, 6

(0, 0, 3, 4, 4, 4, 4, 6) 6 0
(0, 0, 3, 4, 4, 4, 4, 7) 7 0
(0, 0, 3, 4, 4, 4, 4, 8) 8 0
(0, 0, 3, 4, 4, 4, 5, 5) 4 0
(0, 0, 3, 4, 4, 4, 5, 6) 4 0
(0, 0, 3, 4, 4, 4, 5, 7) 7 0
(0, 0, 3, 4, 4, 4, 5, 8) 8 0
(0, 0, 3, 4, 4, 4, 6, 6) 3 0
(0, 0, 3, 4, 4, 4, 6, 7) 3 0
(0, 0, 3, 4, 4, 4, 6, 8) 3 0
(0, 0, 3, 4, 4, 4, 7, 7) 3 0
(0, 0, 3, 4, 4, 4, 7, 8) 3 0
(0, 0, 3, 4, 4, 5, 5, 5) 4 0
(0, 0, 3, 4, 4, 5, 5, 7) 7 0
(0, 0, 3, 4, 4, 5, 5, 8) 8 0
(0, 0, 3, 4, 4, 5, 6, 6) 6 0
(0, 0, 3, 4, 4, 5, 6, 7) 4 0
(0, 0, 3, 4, 4, 5, 6, 8) 8 0
(0, 0, 3, 4, 4, 5, 7, 7) 4 0
(0, 0, 3, 4, 4, 5, 7, 8) 4 0
(0, 0, 3, 4, 4, 5, 8, 8) 4 0
(0, 0, 3, 4, 4, 6, 6, 6) 4 0
(0, 0, 3, 4, 4, 6, 6, 7) 6 0
(0, 0, 3, 4, 4, 6, 6, 8) 4 0
(0, 0, 3, 4, 4, 6, 7, 7) 7 0
(0, 0, 3, 4, 4, 6, 7, 8) 4 8
(0, 0, 3, 4, 4, 6, 8, 8) 4 0
(0, 0, 3, 4, 4, 7, 7, 7) 4 0
(0, 0, 3, 4, 4, 7, 7, 8) 4 0
(0, 0, 3, 4, 4, 7, 8, 8) 4 8
(0, 0, 3, 4, 4, 8, 8, 8) 4 3
(0, 0, 3, 4, 5, 5, 5, 6) 5 0
(0, 0, 3, 4, 5, 5, 5, 7) 7 0
(0, 0, 3, 4, 5, 5, 5, 8) 8 0
(0, 0, 3, 4, 5

...

(1, 2, 5, 5, 5, 6, 8, 8) 6 8
(1, 2, 5, 5, 5, 7, 7, 7) 5 2
(1, 2, 5, 5, 5, 7, 8, 8) 7 8
(1, 2, 5, 5, 5, 8, 8, 8) 5 2
(1, 2, 5, 5, 6, 6, 6, 6) 6 1
(1, 2, 5, 5, 6, 6, 6, 7) 5 1
(1, 2, 5, 5, 6, 6, 7, 8) 6 8
(1, 2, 5, 5, 6, 6, 8, 8) 5 8
(1, 2, 5, 5, 6, 7, 7, 7) 5 1
(1, 2, 5, 5, 6, 7, 7, 8) 7 8
(1, 2, 5, 5, 6, 8, 8, 8) 6 8
(1, 2, 5, 5, 7, 7, 8, 8) 7 8
(1, 2, 5, 5, 7, 8, 8, 8) 7 8
(1, 2, 5, 6, 6, 6, 6, 7) 6 1
(1, 2, 5, 6, 6, 6, 7, 7) 7 1
(1, 2, 5, 6, 6, 6, 8, 8) 5 8
(1, 2, 5, 6, 6, 7, 7, 7) 6 1
(1, 2, 5, 6, 6, 7, 7, 8) 7 8
(1, 2, 5, 6, 6, 7, 8, 8) 6 8
(1, 2, 5, 6, 6, 8, 8, 8) 5 8
(1, 2, 5, 6, 7, 7, 7, 7) 7 1
(1, 2, 5, 6, 7, 7, 8, 8) 7 8
(1, 2, 5, 7, 7, 7, 7, 8) 7 8
(1, 2, 5, 7, 7, 7, 8, 8) 5 8
(1, 2, 5, 7, 7, 8, 8, 8) 5 8
(1, 2, 6, 6, 6, 6, 7, 8) 6 8
(1, 2, 6, 6, 6, 6, 8, 8) 6 8
(1, 2, 6, 6, 6, 7, 7, 7) 6 1
(1, 2, 6, 6, 6, 8, 8, 8) 8 2
(1, 2, 6, 6, 7, 8, 8, 8) 6 8
(1, 2, 6, 6, 8, 8, 8, 8) 8 1
(1, 2, 6, 7, 7, 7, 7, 8) 7 8
(1, 2, 6, 7, 7, 8, 8, 8) 6 8
(1, 2, 6, 7, 8, 8, 8, 8) 8 1
(1, 2, 7, 7, 7

(1, 5, 6, 7, 7, 7, 7, 8) 1 8
(1, 5, 6, 7, 7, 7, 8, 8) 1 8
(1, 5, 6, 7, 7, 8, 8, 8) 1 8
(1, 5, 6, 7, 8, 8, 8, 8) 8 1
(1, 5, 7, 7, 7, 7, 8, 8) 1 8
(1, 5, 7, 7, 7, 8, 8, 8) 1 8
(1, 5, 7, 7, 8, 8, 8, 8) 8 1
(1, 6, 6, 6, 6, 7, 7, 7) 1 6
(1, 6, 6, 6, 6, 7, 7, 8) 1 8
(1, 6, 6, 6, 6, 7, 8, 8) 1 6
(1, 6, 6, 6, 7, 7, 7, 7) 1 7
(1, 6, 6, 6, 7, 7, 7, 8) 1 8
(1, 6, 6, 6, 7, 7, 8, 8) 1 8
(1, 6, 6, 6, 7, 8, 8, 8) 1 8
(1, 6, 6, 7, 7, 7, 7, 8) 1 8
(1, 6, 6, 7, 7, 7, 8, 8) 1 8
(1, 6, 7, 7, 7, 7, 8, 8) 1 7
(1, 6, 7, 7, 7, 8, 8, 8) 1 8
(2, 2, 2, 2, 3, 3, 3, 5) 5 2
(2, 2, 2, 2, 3, 3, 3, 6) 6 2
(2, 2, 2, 2, 3, 3, 3, 7) 7 2
(2, 2, 2, 2, 3, 3, 3, 8) 8 2
(2, 2, 2, 2, 3, 3, 4, 5) 2 5
(2, 2, 2, 2, 3, 3, 5, 5) 3 2
(2, 2, 2, 2, 3, 3, 6, 6) 3 2
(2, 2, 2, 2, 3, 3, 6, 7) 2 7
(2, 2, 2, 2, 3, 3, 7, 7) 3 2
(2, 2, 2, 2, 3, 3, 8, 8) 3 8
(2, 2, 2, 2, 3, 4, 4, 5) 2 5
(2, 2, 2, 2, 3, 4, 5, 7) 2 7
(2, 2, 2, 2, 3, 4, 5, 8) 2 8
(2, 2, 2, 2, 3, 4, 6, 7) 2 7
(2, 2, 2, 2, 3, 5, 6, 6) 5 2
(2, 2, 2, 2, 3, 5, 7, 7) 5 2
(2, 2, 2, 2, 3

(2, 2, 4, 5, 5, 5, 7, 8) 4 8
(2, 2, 4, 5, 5, 5, 8, 8) 2 8
(2, 2, 4, 5, 5, 6, 6, 6) 6 4
(2, 2, 4, 5, 5, 6, 7, 7) 7 4
(2, 2, 4, 5, 5, 6, 8, 8) 5 8
(2, 2, 4, 5, 5, 7, 7, 7) 5 4
(2, 2, 4, 5, 5, 7, 8, 8) 7 8
(2, 2, 4, 5, 5, 8, 8, 8) 5 8
(2, 2, 4, 5, 6, 6, 6, 7) 6 7
(2, 2, 4, 5, 6, 6, 7, 7) 7 4
(2, 2, 4, 5, 6, 6, 8, 8) 6 8
(2, 2, 4, 5, 6, 7, 7, 8) 7 8
(2, 2, 4, 5, 7, 8, 8, 8) 7 8
(2, 2, 4, 6, 6, 6, 6, 7) 4 6
(2, 2, 4, 6, 6, 6, 7, 8) 4 8
(2, 2, 4, 6, 6, 6, 8, 8) 2 8
(2, 2, 4, 6, 6, 7, 7, 7) 2 4
(2, 2, 4, 6, 6, 7, 7, 8) 4 8
(2, 2, 4, 6, 6, 7, 8, 8) 8 7
(2, 2, 4, 6, 6, 8, 8, 8) 4 8
(2, 2, 4, 6, 7, 7, 7, 7) 4 7
(2, 2, 4, 6, 7, 7, 8, 8) 4 8
(2, 2, 4, 6, 7, 8, 8, 8) 4 8
(2, 2, 4, 7, 7, 7, 7, 8) 4 7
(2, 2, 4, 7, 7, 7, 8, 8) 2 8
(2, 2, 4, 7, 7, 8, 8, 8) 4 8
(2, 2, 4, 7, 8, 8, 8, 8) 4 8
(2, 2, 5, 5, 5, 5, 6, 6) 6 5
(2, 2, 5, 5, 5, 5, 8, 8) 5 8
(2, 2, 5, 5, 5, 6, 6, 7) 6 7
(2, 2, 5, 5, 5, 6, 7, 7) 7 6
(2, 2, 5, 5, 5, 6, 8, 8) 6 8
(2, 2, 5, 5, 5, 7, 8, 8) 7 8
(2, 2, 5, 5, 6, 6, 6, 6) 5 6
(2, 2, 5, 5, 6

(2, 5, 6, 6, 7, 7, 7, 8) 2 8
(2, 5, 6, 6, 7, 7, 8, 8) 2 5
(2, 5, 6, 6, 7, 8, 8, 8) 2 8
(2, 5, 6, 7, 7, 7, 7, 8) 2 8
(2, 5, 6, 7, 7, 7, 8, 8) 2 8
(2, 5, 6, 7, 7, 8, 8, 8) 2 8
(2, 5, 6, 7, 8, 8, 8, 8) 8 2
(2, 5, 7, 7, 7, 7, 8, 8) 2 8
(2, 5, 7, 7, 7, 8, 8, 8) 2 8
(2, 6, 6, 6, 6, 7, 7, 7) 2 6
(2, 6, 6, 6, 6, 7, 7, 8) 2 8
(2, 6, 6, 6, 6, 7, 8, 8) 2 8
(2, 6, 6, 6, 7, 7, 7, 7) 2 7
(2, 6, 6, 6, 7, 7, 7, 8) 2 8
(2, 6, 6, 6, 7, 7, 8, 8) 2 8
(2, 6, 6, 6, 7, 8, 8, 8) 2 8
(2, 6, 6, 7, 7, 7, 7, 8) 2 8
(2, 6, 6, 7, 7, 7, 8, 8) 2 8
(2, 6, 7, 7, 7, 7, 8, 8) 2 7
(2, 6, 7, 7, 7, 8, 8, 8) 2 8
(3, 3, 3, 3, 4, 4, 4, 6) 6 3
(3, 3, 3, 3, 4, 4, 4, 7) 7 3
(3, 3, 3, 3, 4, 4, 4, 8) 8 3
(3, 3, 3, 3, 4, 4, 5, 6) 3 6
(3, 3, 3, 3, 4, 4, 6, 6) 4 3
(3, 3, 3, 3, 4, 4, 6, 7) 3 7
(3, 3, 3, 3, 4, 4, 6, 8) 3 8
(3, 3, 3, 3, 4, 4, 7, 7) 4 3
(3, 3, 3, 3, 4, 4, 8, 8) 4 8
(3, 3, 3, 3, 4, 5, 5, 6) 3 6
(3, 3, 3, 3, 4, 5, 6, 7) 3 7
(3, 3, 3, 3, 4, 5, 6, 8) 3 8
(3, 3, 3, 3, 4, 6, 7, 7) 6 3
(3, 3, 3, 3, 4, 6, 7, 8) 3 8
(3, 3, 3, 3, 4

(4, 4, 4, 4, 6, 6, 6, 8) 4 8
(4, 4, 4, 4, 6, 6, 7, 7) 6 4
(4, 4, 4, 4, 6, 6, 7, 8) 4 8
(4, 4, 4, 4, 6, 6, 8, 8) 6 8
(4, 4, 4, 4, 6, 7, 7, 8) 4 8
(4, 4, 4, 4, 6, 7, 8, 8) 4 8
(4, 4, 4, 4, 6, 8, 8, 8) 4 8
(4, 4, 4, 4, 7, 7, 7, 8) 4 8
(4, 4, 4, 4, 7, 8, 8, 8) 4 8
(4, 4, 4, 4, 8, 8, 8, 8) 4 8
(4, 4, 4, 5, 5, 5, 5, 7) 7 5
(4, 4, 4, 5, 5, 5, 5, 8) 8 5
(4, 4, 4, 5, 5, 6, 7, 7) 7 6
(4, 4, 4, 5, 5, 6, 8, 8) 4 8
(4, 4, 4, 5, 5, 7, 8, 8) 7 8
(4, 4, 4, 5, 6, 6, 7, 7) 7 5
(4, 4, 4, 5, 6, 6, 8, 8) 6 8
(4, 4, 4, 5, 6, 7, 7, 8) 7 8
(4, 4, 4, 5, 7, 7, 8, 8) 7 8
(4, 4, 4, 5, 7, 8, 8, 8) 7 8
(4, 4, 4, 6, 6, 6, 6, 8) 6 8
(4, 4, 4, 6, 6, 7, 7, 8) 7 8
(4, 4, 4, 6, 6, 7, 8, 8) 6 7
(4, 4, 4, 6, 7, 7, 8, 8) 7 8
(4, 4, 4, 7, 7, 7, 7, 8) 7 8
(4, 4, 5, 5, 5, 5, 7, 7) 4 5
(4, 4, 5, 5, 5, 5, 8, 8) 4 8
(4, 4, 5, 5, 5, 6, 6, 7) 6 7
(4, 4, 5, 5, 5, 6, 7, 7) 5 6
(4, 4, 5, 5, 5, 6, 8, 8) 5 8
(4, 4, 5, 5, 5, 7, 8, 8) 7 8
(4, 4, 5, 5, 6, 7, 7, 7) 4 6
(4, 4, 5, 5, 6, 7, 8, 8) 4 8
(4, 4, 5, 5, 6, 8, 8, 8) 5 8
(4, 4, 5, 5, 7

In [17]:
for i, val in enumerate(normal_value_hand_nml09):
    print (val, hand_nml[i])

0.6973367750723577 (0, 0, 0, 0, 1, 1, 1)
0.7246309856034419 (0, 0, 0, 0, 1, 1, 2)
0.6545183580235789 (0, 0, 0, 0, 1, 1, 3)
0.6519950092562166 (0, 0, 0, 0, 1, 1, 4)
0.6568514705488455 (0, 0, 0, 0, 1, 1, 5)
0.6598905964169361 (0, 0, 0, 0, 1, 1, 6)
0.6346042060867648 (0, 0, 0, 0, 1, 1, 7)
0.6120785458098912 (0, 0, 0, 0, 1, 1, 8)
0.7246309856034419 (0, 0, 0, 0, 1, 2, 2)
0.741486409731981 (0, 0, 0, 0, 1, 2, 3)
0.719707089205553 (0, 0, 0, 0, 1, 2, 4)
0.7105429943599149 (0, 0, 0, 0, 1, 2, 5)
0.7100152909639701 (0, 0, 0, 0, 1, 2, 6)
0.6961360587490051 (0, 0, 0, 0, 1, 2, 7)
0.6848277990487752 (0, 0, 0, 0, 1, 2, 8)
0.6716685086947612 (0, 0, 0, 0, 1, 3, 3)
0.6620630420423924 (0, 0, 0, 0, 1, 3, 4)
0.6315133496961725 (0, 0, 0, 0, 1, 3, 5)
0.6175813707095889 (0, 0, 0, 0, 1, 3, 6)
0.6100154382944187 (0, 0, 0, 0, 1, 3, 7)
0.6008127649821067 (0, 0, 0, 0, 1, 3, 8)
0.6829530767279606 (0, 0, 0, 0, 1, 4, 4)
0.6553962708531743 (0, 0, 0, 0, 1, 4, 5)
0.6308434645601146 (0, 0, 0, 0, 1, 4, 6)
0.6053722198130357

0.6302344422945003 (0, 1, 1, 2, 3, 6, 8)
0.6837873169601981 (0, 1, 1, 2, 3, 7, 7)
0.6152650965509476 (0, 1, 1, 2, 3, 7, 8)
0.677478147530938 (0, 1, 1, 2, 3, 8, 8)
0.7378489382487667 (0, 1, 1, 2, 4, 4, 4)
0.6635824218519538 (0, 1, 1, 2, 4, 4, 5)
0.6634773709480274 (0, 1, 1, 2, 4, 4, 6)
0.6511354676188857 (0, 1, 1, 2, 4, 4, 7)
0.6315386380777532 (0, 1, 1, 2, 4, 4, 8)
0.6393110546196116 (0, 1, 1, 2, 4, 5, 5)
0.6876095867648268 (0, 1, 1, 2, 4, 5, 6)
0.6344550885797687 (0, 1, 1, 2, 4, 5, 7)
0.6195788595136269 (0, 1, 1, 2, 4, 5, 8)
0.6431802621598486 (0, 1, 1, 2, 4, 6, 6)
0.6532360709862804 (0, 1, 1, 2, 4, 6, 7)
0.6161661543766229 (0, 1, 1, 2, 4, 6, 8)
0.6519876626241632 (0, 1, 1, 2, 4, 7, 7)
0.5937633986567223 (0, 1, 1, 2, 4, 7, 8)
0.6447084386138645 (0, 1, 1, 2, 4, 8, 8)
0.7484386679926146 (0, 1, 1, 2, 5, 5, 5)
0.6489880616820531 (0, 1, 1, 2, 5, 5, 6)
0.6332517099540066 (0, 1, 1, 2, 5, 5, 7)
0.6266358755343233 (0, 1, 1, 2, 5, 5, 8)
0.6462933992161031 (0, 1, 1, 2, 5, 6, 6)
0.678714852550341

0.723343337941014 (0, 3, 4, 5, 8, 8, 8)
0.661339883394073 (0, 3, 4, 6, 6, 6, 6)
0.7085014885624338 (0, 3, 4, 6, 6, 6, 7)
0.6878120615741482 (0, 3, 4, 6, 6, 6, 8)
0.6183213655940885 (0, 3, 4, 6, 6, 7, 7)
0.629260665608777 (0, 3, 4, 6, 6, 7, 8)
0.6116043566886988 (0, 3, 4, 6, 6, 8, 8)
0.717142446105704 (0, 3, 4, 6, 7, 7, 7)
0.6195788595136269 (0, 3, 4, 6, 7, 7, 8)
0.6358319217695432 (0, 3, 4, 6, 7, 8, 8)
0.6884615285856426 (0, 3, 4, 6, 8, 8, 8)
0.6414475777592983 (0, 3, 4, 7, 7, 7, 7)
0.6733667810647618 (0, 3, 4, 7, 7, 7, 8)
0.5938004945376468 (0, 3, 4, 7, 7, 8, 8)
0.6654399365868359 (0, 3, 4, 7, 8, 8, 8)
0.6238169123580775 (0, 3, 4, 8, 8, 8, 8)
0.6887595868141327 (0, 3, 5, 5, 5, 5, 6)
0.6455069664718285 (0, 3, 5, 5, 5, 5, 7)
0.6215563545603056 (0, 3, 5, 5, 5, 5, 8)
0.6585250093235743 (0, 3, 5, 5, 5, 6, 6)
0.6864956172823597 (0, 3, 5, 5, 5, 6, 7)
0.6373423819056141 (0, 3, 5, 5, 5, 6, 8)
0.6543864554889072 (0, 3, 5, 5, 5, 7, 7)
0.6222058233737389 (0, 3, 5, 5, 5, 7, 8)
0.6436717342984738 (

0.8181817895840537 (1, 2, 3, 3, 4, 5, 6)
0.7285646407592027 (1, 2, 3, 3, 4, 5, 7)
0.7285646407592027 (1, 2, 3, 3, 4, 5, 8)
0.7747951904394039 (1, 2, 3, 3, 4, 6, 6)
0.6989518392233042 (1, 2, 3, 3, 4, 6, 7)
0.6733679626295341 (1, 2, 3, 3, 4, 6, 8)
0.7658200923045007 (1, 2, 3, 3, 4, 7, 7)
0.6479989279473912 (1, 2, 3, 3, 4, 7, 8)
0.762792999213785 (1, 2, 3, 3, 4, 8, 8)
0.8447580636141985 (1, 2, 3, 3, 5, 5, 5)
0.6889090442503292 (1, 2, 3, 3, 5, 5, 6)
0.6666419703034561 (1, 2, 3, 3, 5, 5, 7)
0.6626700924378226 (1, 2, 3, 3, 5, 5, 8)
0.7302155666492003 (1, 2, 3, 3, 5, 6, 6)
0.751718768877801 (1, 2, 3, 3, 5, 6, 7)
0.6771350328267706 (1, 2, 3, 3, 5, 6, 8)
0.7077771256097285 (1, 2, 3, 3, 5, 7, 7)
0.634116243369854 (1, 2, 3, 3, 5, 7, 8)
0.7118985074447143 (1, 2, 3, 3, 5, 8, 8)
0.7978808427161326 (1, 2, 3, 3, 6, 6, 6)
0.6819253511451081 (1, 2, 3, 3, 6, 6, 7)
0.6682294014517687 (1, 2, 3, 3, 6, 6, 8)
0.6794148920806168 (1, 2, 3, 3, 6, 7, 7)
0.7362821582914815 (1, 2, 3, 3, 6, 7, 8)
0.666368152852562 (

0.6712554807882033 (2, 4, 4, 7, 7, 7, 7)
0.6644837825624351 (2, 4, 4, 7, 7, 7, 8)
0.5833053342798773 (2, 4, 4, 7, 7, 8, 8)
0.6699169003361392 (2, 4, 4, 7, 8, 8, 8)
0.6530499795545719 (2, 4, 4, 8, 8, 8, 8)
0.7851833208544307 (2, 4, 5, 5, 5, 5, 6)
0.6922747251936507 (2, 4, 5, 5, 5, 5, 7)
0.6741876393622154 (2, 4, 5, 5, 5, 5, 8)
0.7303670635551626 (2, 4, 5, 5, 5, 6, 6)
0.7454979761608784 (2, 4, 5, 5, 5, 6, 7)
0.7022962329843601 (2, 4, 5, 5, 5, 6, 8)
0.7201448648684493 (2, 4, 5, 5, 5, 7, 7)
0.6630101018090961 (2, 4, 5, 5, 5, 7, 8)
0.7139335545548668 (2, 4, 5, 5, 5, 8, 8)
0.7370551008750548 (2, 4, 5, 5, 6, 6, 6)
0.6961121719131333 (2, 4, 5, 5, 6, 6, 7)
0.6473038376454908 (2, 4, 5, 5, 6, 6, 8)
0.6900532135361743 (2, 4, 5, 5, 6, 7, 7)
0.7281754157218496 (2, 4, 5, 5, 6, 7, 8)
0.6968138144853288 (2, 4, 5, 5, 6, 8, 8)
0.7423214859956736 (2, 4, 5, 5, 7, 7, 7)
0.6109960939703766 (2, 4, 5, 5, 7, 7, 8)
0.6168833337943709 (2, 4, 5, 5, 7, 8, 8)
0.7288157631508442 (2, 4, 5, 5, 8, 8, 8)
0.77835006073684

In [20]:
for i, val in enumerate(value_hand_nml09):
    print (val, hand_nml[i])

1.0849632785146 (0, 0, 0, 0, 1, 1, 1)
1.8918918796326243 (0, 0, 0, 0, 1, 1, 2)
1.1723553832002203 (0, 0, 0, 0, 1, 1, 3)
1.1663053926654776 (0, 0, 0, 0, 1, 1, 4)
1.169458742303457 (0, 0, 0, 0, 1, 1, 5)
1.166840864490913 (0, 0, 0, 0, 1, 1, 6)
1.1653818893978962 (0, 0, 0, 0, 1, 1, 7)
1.2025159909404564 (0, 0, 0, 0, 1, 1, 8)
1.8918918796326243 (0, 0, 0, 0, 1, 2, 2)
1.2687220037578615 (0, 0, 0, 0, 1, 2, 3)
1.2687220037578615 (0, 0, 0, 0, 1, 2, 4)
1.2687220037578615 (0, 0, 0, 0, 1, 2, 5)
1.2687220037578615 (0, 0, 0, 0, 1, 2, 6)
1.2687220037578615 (0, 0, 0, 0, 1, 2, 7)
1.474201455812656 (0, 0, 0, 0, 1, 2, 8)
1.0524813906953916 (0, 0, 0, 0, 1, 3, 3)
1.02003640603508 (0, 0, 0, 0, 1, 3, 4)
1.02200980358595 (0, 0, 0, 0, 1, 3, 5)
1.0222840262253536 (0, 0, 0, 0, 1, 3, 6)
1.0212042165797268 (0, 0, 0, 0, 1, 3, 7)
1.0560069637082223 (0, 0, 0, 0, 1, 3, 8)
1.0434276578966497 (0, 0, 0, 0, 1, 4, 4)
1.0227200187226777 (0, 0, 0, 0, 1, 4, 5)
1.021490739448774 (0, 0, 0, 0, 1, 4, 6)
1.0150238407095313 (0, 0, 0

1.4371219829258324 (0, 1, 1, 2, 6, 6, 7)
1.3729556817733726 (0, 1, 1, 2, 6, 6, 8)
1.4355291043442189 (0, 1, 1, 2, 6, 7, 7)
1.234961379455687 (0, 1, 1, 2, 6, 7, 8)
1.3034562630931146 (0, 1, 1, 2, 6, 8, 8)
1.6079489238756937 (0, 1, 1, 2, 7, 7, 7)
1.3605370522117337 (0, 1, 1, 2, 7, 7, 8)
1.2999770601753124 (0, 1, 1, 2, 7, 8, 8)
1.2767754328756422 (0, 1, 1, 2, 8, 8, 8)
1.6296209304810183 (0, 1, 1, 3, 3, 3, 3)
1.6887501901393238 (0, 1, 1, 3, 3, 3, 4)
1.715915584618613 (0, 1, 1, 3, 3, 3, 5)
1.6856395947335951 (0, 1, 1, 3, 3, 3, 6)
1.658430333950323 (0, 1, 1, 3, 3, 3, 7)
1.4966995448579394 (0, 1, 1, 3, 3, 3, 8)
1.9168018476072313 (0, 1, 1, 3, 3, 4, 4)
1.670330451005246 (0, 1, 1, 3, 3, 4, 5)
1.539108744362792 (0, 1, 1, 3, 3, 4, 6)
1.5191230958646629 (0, 1, 1, 3, 3, 4, 7)
1.4524182488954316 (0, 1, 1, 3, 3, 4, 8)
1.925630161859986 (0, 1, 1, 3, 3, 5, 5)
1.566209316644095 (0, 1, 1, 3, 3, 5, 6)
1.5455010074927913 (0, 1, 1, 3, 3, 5, 7)
1.4689703507347691 (0, 1, 1, 3, 3, 5, 8)
1.820220687559691 (0, 1

1.3480655247441944 (0, 3, 4, 4, 4, 6, 8)
1.6930914088237308 (0, 3, 4, 4, 4, 7, 7)
1.3286024541348742 (0, 3, 4, 4, 4, 7, 8)
1.2144412078790696 (0, 3, 4, 4, 4, 8, 8)
1.759086470823338 (0, 3, 4, 4, 5, 5, 5)
1.6481007026311636 (0, 3, 4, 4, 5, 5, 6)
1.6954922990808468 (0, 3, 4, 4, 5, 5, 7)
1.544679932625941 (0, 3, 4, 4, 5, 5, 8)
1.6658309147854524 (0, 3, 4, 4, 5, 6, 6)
1.4985249731147716 (0, 3, 4, 4, 5, 6, 7)
1.3773946158749615 (0, 3, 4, 4, 5, 6, 8)
1.6561864113533056 (0, 3, 4, 4, 5, 7, 7)
1.3723138618198263 (0, 3, 4, 4, 5, 7, 8)
1.251247026894867 (0, 3, 4, 4, 5, 8, 8)
1.7065345416084565 (0, 3, 4, 4, 6, 6, 6)
1.504140217504591 (0, 3, 4, 4, 6, 6, 7)
1.4518993829989435 (0, 3, 4, 4, 6, 6, 8)
1.5192973915274455 (0, 3, 4, 4, 6, 7, 7)
1.2907745707008114 (0, 3, 4, 4, 6, 7, 8)
1.2043130805249707 (0, 3, 4, 4, 6, 8, 8)
1.677559190556357 (0, 3, 4, 4, 7, 7, 7)
1.4469854119886485 (0, 3, 4, 4, 7, 7, 8)
1.1961627825696308 (0, 3, 4, 4, 7, 8, 8)
1.1087858945744822 (0, 3, 4, 4, 8, 8, 8)
1.5622942716524835 (0

1.4088478164210698 (1, 2, 3, 4, 4, 6, 8)
1.7379343170791264 (1, 2, 3, 4, 4, 7, 7)
1.371142094564099 (1, 2, 3, 4, 4, 7, 8)
1.2869245274632215 (1, 2, 3, 4, 4, 8, 8)
1.6655412659905353 (1, 2, 3, 4, 5, 5, 5)
1.5750403753937507 (1, 2, 3, 4, 5, 5, 6)
1.505009656142782 (1, 2, 3, 4, 5, 5, 7)
1.455875254499399 (1, 2, 3, 4, 5, 5, 8)
1.5439554259208168 (1, 2, 3, 4, 5, 6, 6)
1.6363636363635456 (1, 2, 3, 4, 5, 6, 7)
1.3748043978825 (1, 2, 3, 4, 5, 6, 8)
1.5709487885663318 (1, 2, 3, 4, 5, 7, 7)
1.3682869022765363 (1, 2, 3, 4, 5, 7, 8)
1.168328130943427 (1, 2, 3, 4, 5, 8, 8)
1.7260298511881387 (1, 2, 3, 4, 6, 6, 6)
1.4621323926165377 (1, 2, 3, 4, 6, 6, 7)
1.4338292814806046 (1, 2, 3, 4, 6, 6, 8)
1.5456294741286927 (1, 2, 3, 4, 6, 7, 7)
1.2465469233983995 (1, 2, 3, 4, 6, 7, 8)
1.1810282975372604 (1, 2, 3, 4, 6, 8, 8)
1.6240859938666918 (1, 2, 3, 4, 7, 7, 7)
1.4371905945802466 (1, 2, 3, 4, 7, 7, 8)
1.2021908167348676 (1, 2, 3, 4, 7, 8, 8)
1.050270180285249 (1, 2, 3, 4, 8, 8, 8)
1.4952001703205944 (1, 2

1.3898350305153933 (2, 3, 4, 5, 5, 7, 8)
1.305801297585123 (2, 3, 4, 5, 5, 8, 8)
1.8681318681318677 (2, 3, 4, 5, 6, 6, 6)
1.5627337243869723 (2, 3, 4, 5, 6, 6, 7)
1.5011627546174222 (2, 3, 4, 5, 6, 6, 8)
1.6979669850248207 (2, 3, 4, 5, 6, 7, 7)
1.2940858176107009 (2, 3, 4, 5, 6, 7, 8)
1.1555659455660896 (2, 3, 4, 5, 6, 8, 8)
1.7323254013542289 (2, 3, 4, 5, 7, 7, 7)
1.5011691754729102 (2, 3, 4, 5, 7, 7, 8)
1.2155051731460316 (2, 3, 4, 5, 7, 8, 8)
1.0571400162190525 (2, 3, 4, 5, 8, 8, 8)
1.5429462076163778 (2, 3, 4, 6, 6, 6, 6)
1.5982545479127204 (2, 3, 4, 6, 6, 6, 7)
1.4365695049160996 (2, 3, 4, 6, 6, 6, 8)
1.5636187120738732 (2, 3, 4, 6, 6, 7, 7)
1.3561230110508387 (2, 3, 4, 6, 6, 7, 8)
1.3017285106983245 (2, 3, 4, 6, 6, 8, 8)
1.5831933219509586 (2, 3, 4, 6, 7, 7, 7)
1.3982982399291728 (2, 3, 4, 6, 7, 7, 8)
1.1511300588886397 (2, 3, 4, 6, 7, 8, 8)
1.0775195307259806 (2, 3, 4, 6, 8, 8, 8)
1.4926899188289042 (2, 3, 4, 7, 7, 7, 7)
1.393525091689798 (2, 3, 4, 7, 7, 7, 8)
1.3074392082748751